<a href="https://colab.research.google.com/github/radhikasethi2011/ProteinClassify/blob/main/colab/pdb_ca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/radhikasethi2011/ProteinClassify.git

Cloning into 'ProteinClassify'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [2]:
%pip install BioPython
import pandas as pd
import os
import glob as glob
from Bio.PDB import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 4.4 MB/s 


In [3]:
cols = ['domid','pdbid', 'pdbchain']
scop = pd.read_csv('/content/ProteinClassify/scop-represented-structures-latest.txt', header = None, skiprows=6, names=cols, delimiter = ' ')
scop

,domid,pdbid,pdbchain
0,8000061,2DT5,B
1,8000376,2FR1,A
2,8000376,6W7S,A
3,8000376,6WH9,A
4,8000376,6WH9,D
...,...,...,...
858211,8107712,7T7E,A
858212,8107712,7T7F,A
858213,8107712,7T7G,A
858214,8107715,7R1K,A


https://files.rcsb.org/download/2FR1.pdb


In [4]:
%mkdir /content/pdb_files_download
%mkdir /content/pdb_ca_output
%mkdir /content/pdb_ca_csv

running for first 5 

In [5]:
for i in range(5):
  pd_id = scop['pdbid'][i]
  pd_chain = scop['pdbchain'][i]
  !wget -P /content/pdb_files_download  https://files.rcsb.org/download/{pd_id}.pdb -O /content/pdb_files_download/{pd_id}_{pd_chain}.pdb

--2022-06-04 08:46:44--  https://files.rcsb.org/download/2DT5.pdb
Resolving files.rcsb.org (files.rcsb.org)... 132.249.210.134
Connecting to files.rcsb.org (files.rcsb.org)|132.249.210.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘/content/pdb_files_download/2DT5_B.pdb’

2DT5_B.pdb              [        <=>         ] 340.06K   167KB/s    in 2.0s    

2022-06-04 08:46:47 (167 KB/s) - ‘/content/pdb_files_download/2DT5_B.pdb’ saved [348219]

--2022-06-04 08:46:47--  https://files.rcsb.org/download/2FR1.pdb
Resolving files.rcsb.org (files.rcsb.org)... 132.249.210.134
Connecting to files.rcsb.org (files.rcsb.org)|132.249.210.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘/content/pdb_files_download/2FR1_A.pdb’

2FR1_A.pdb              [        <=>         ] 328.59K   154KB/s    in 2.1s    

2022-06-04 08:46:50 (154 KB/s) - ‘/con

The first element "atom" means this relates to an actual atom in the protein. The 1 relates to a general count, N relates to the type of atom, "MET" is the name of the residue, "A" relates to the the type of chain, 1 (the second "1") is the atom count and then the next 3 numbers are the x-y-z positions in space.



#Understanding

atom | count | type of atom | ressidue name | chain | atom count | x | y  | z
--- | --- | --- | --- | --- | --- | --- | --- | ---
ATOM | 1 | N | MET | A | 1 | 63.645 | 97.355 | 31.526 | 1.00 | 33.80 | N

In [6]:
#parser=PDBParser(PERMISSIVE = True, QUIET = True)
#data = parser.get_structure('test', '/content/pdb_files/2FR1_A.pdb')
#print("data: ", data)
#print("data type: ", type(data))
#print("data keys: ", data.header.keys())
#print("data name: ", data.header["name"]) 
#print("release date: ", data.header["release_date"])
#print("resolution: ",data.header["resolution"])
#model = data.get_models()
#models = list(model)
#chains = list(models[0].get_chains()) 
#print("Chains: ", list(models[0].get_chains()))
#residue = list(chains[0].get_residues())
#print("len residue: ", len(residue))
#atoms = list(residue[0].get_atoms()) 
#print("atoms: ", atoms)

In [8]:
def parse_pdb(pdbpath):
  parser=PDBParser()
  io = PDBIO()
  data = parser.get_structure('test', pdbpath)
  data_resolution = data.header["resolution"]
  data_keywords = data.header["keywords"]

  print("Sample name: " + str(data))
  print("Resolution: " + str(data_resolution))
  print("Keywords: " + str(data_keywords))
  print("Model: " + str(data[0]))

  #initialize IO 
  io=PDBIO()

  #custom select
  class Select():
      def accept_model(self, model):
          return True
      def accept_chain(self, chain):
          return True
      def accept_residue(self, residue):
          return True       
      def accept_atom(self, atom):
          #print("atom id:" + atom.get_id())
          #print("atom name:" + atom.get_name())
          if atom.get_name() == 'CA':   
              return True
          else:
              return False

  #write to output file
  io.set_structure(data)
  filename = pdbpath.split('/')[3].split('.')[0]
  io.save(f"/content/pdb_ca_output/{filename}.txt", Select())
  return f'/content/pdb_ca_output/{filename}.txt'

PDB parsing and extraction of the following items:
Amino acid, chain, position number, CA, x, y, z


In [14]:
#str1 = '/content/pdb_files_download/6WH9_A.pdb'
#str1.split('/')[3].split('.')[0]

'6WH9_A'

In [15]:
#str1 = '/content/pdb_ca_output/6WH9_A.pdb'
#str1.split('/')[3].split('.')[0]

'6WH9_A'

In [11]:
def clean_pdb(pdb_txt):
  cols = ['atom','general_count', 'type', 'amino', 'chain', 'x','y','z', '1', '2', '3']
  ca_df = pd.read_csv(pdb_txt, header = None, names=cols, delim_whitespace=True)
  ca_df['chain'], ca_df['position'] = ca_df['chain'].str[:1], ca_df['chain'].str[1:]
  ca_df.drop(columns=['atom', 'general_count','1','2','3'], inplace=True)
  ca_df.dropna(inplace=True)
  filename = pdb_txt.split('/')[3].split('.')[0]
  ca_df.to_csv(f'/content/pdb_ca_csv/{filename}_ca.csv')
  ca_df

In [13]:
pdb_files_download_list = glob.glob('/content/pdb_files_download/*')
#pdb_files_download_list
for i in range(len(pdb_files_download_list)):
  print("i: ", i)
  pdb_txt = parse_pdb(pdb_files_download_list[i])
  clean_pdb(pdb_txt)

i:  0


/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain G is discontinuous at line 41055.
  PDBConstructionWarning,
/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 41060.
  PDBConstructionWarning,
/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain F is discontinuous at line 41065.
  PDBConstructionWarning,
/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 41070.
  PDBConstructionWarning,
/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 41075.
  PDBConstructionWarning,
/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 41080.
  PDBC

Sample name: <Structure id=test>
Resolution: 2.75
Keywords: ketoreductase, antibody fragment, oxidoreductase-immune system complex
Model: <Model id=0>
i:  1


/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain G is discontinuous at line 41055.
  PDBConstructionWarning,
/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 41060.
  PDBConstructionWarning,
/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain F is discontinuous at line 41065.
  PDBConstructionWarning,
/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 41070.
  PDBConstructionWarning,
/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 41075.
  PDBConstructionWarning,
/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 41080.
  PDBC

Sample name: <Structure id=test>
Resolution: 2.75
Keywords: ketoreductase, antibody fragment, oxidoreductase-immune system complex
Model: <Model id=0>
i:  2
Sample name: <Structure id=test>
Resolution: 1.79
Keywords: short chain dehydrogenase/reductase, oxidoreductase
Model: <Model id=0>
i:  3
Sample name: <Structure id=test>
Resolution: 2.16
Keywords: rex, nadh, nad, rossmann fold, redox sensing, winged helix, thermus themophilus, riken structural genomics/proteomics initiative, rsgi, structural genomics, dna binding protein
Model: <Model id=0>


/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3757.
  PDBConstructionWarning,
/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3828.
  PDBConstructionWarning,
/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3882.
  PDBConstructionWarning,
/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4063.
  PDBConstructionWarning,


AttributeError: ignored